In [43]:
# from pyspark.sql import SparkSession
from pyspark.sql.functions import col,max,avg,sum

In [2]:
spark=SparkSession.builder.appName("groupby.com").getOrCreate()


24/09/11 11:41:10 WARN Utils: Your hostname, AI-CJB-LAP-459 resolves to a loopback address: 127.0.1.1; using 192.168.1.164 instead (on interface wlp0s20f3)
24/09/11 11:41:10 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/11 11:41:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/09/11 11:41:11 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/09/11 11:41:11 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
data = [("James","Sales","NY",90000,34,10000),
    ("Michael","Sales","NY",86000,56,20000),
    ("Robert","Sales","CA",81000,30,23000),
    ("Maria","Finance","CA",90000,24,23000),
    ("Raman","Finance","CA",99000,40,24000),
    ("Scott","Finance","NY",83000,36,19000),
    ("Jen","Finance","NY",79000,53,15000),
    ("Jeff","Marketing","CA",80000,25,18000),
    ("Kumar","Marketing","NY",91000,50,21000)
  ]

In [55]:
columns=["name","department","state","salary","age","hike"]
df=spark.createDataFrame(data=data,schema=columns)
df.show(5)

+-------+----------+-----+------+---+-----+
|   name|department|state|salary|age| hike|
+-------+----------+-----+------+---+-----+
|  James|     Sales|   NY| 90000| 34|10000|
|Michael|     Sales|   NY| 86000| 56|20000|
| Robert|     Sales|   CA| 81000| 30|23000|
|  Maria|   Finance|   CA| 90000| 24|23000|
|  Raman|   Finance|   CA| 99000| 40|24000|
+-------+----------+-----+------+---+-----+
only showing top 5 rows



In [6]:
df.groupBy("department").sum("salary").show()

+----------+-----------+
|department|sum(salary)|
+----------+-----------+
|     Sales|     257000|
|   Finance|     351000|
| Marketing|     171000|
+----------+-----------+



In [8]:
df.groupBy("department").count().show()

+----------+-----+
|department|count|
+----------+-----+
|     Sales|    3|
|   Finance|    4|
| Marketing|    2|
+----------+-----+



suming up two columns salary and hike

In [23]:
sum_sal_hike=df.withColumn("revised salary",col("salary")+col("hike"))
new =sum_sal_hike.groupby('department').agg(max("revised salary").alias("revised_salary"))
# new.show()
sum_sal_hike.repartition(5)

+-------+----------+-----+------+---+-----+--------------+
|   name|department|state|salary|age| hike|revised salary|
+-------+----------+-----+------+---+-----+--------------+
|  James|     Sales|   NY| 90000| 34|10000|        100000|
|Michael|     Sales|   NY| 86000| 56|20000|        106000|
| Robert|     Sales|   CA| 81000| 30|23000|        104000|
|  Maria|   Finance|   CA| 90000| 24|23000|        113000|
|  Raman|   Finance|   CA| 99000| 40|24000|        123000|
|  Scott|   Finance|   NY| 83000| 36|19000|        102000|
|    Jen|   Finance|   NY| 79000| 53|15000|         94000|
|   Jeff| Marketing|   CA| 80000| 25|18000|         98000|
|  Kumar| Marketing|   NY| 91000| 50|21000|        112000|
+-------+----------+-----+------+---+-----+--------------+



In [26]:
sum_sal_hike.groupBy("state").avg("revised salary").show()
sum_sal_hike.groupBy("department").mean("revised salary").show()

+-----+-------------------+
|state|avg(revised salary)|
+-----+-------------------+
|   NY|           102800.0|
|   CA|           109500.0|
+-----+-------------------+

+----------+-------------------+
|department|avg(revised salary)|
+----------+-------------------+
|     Sales| 103333.33333333333|
|   Finance|           108000.0|
| Marketing|           105000.0|
+----------+-------------------+



USING GROUP BY WITH MULTIPLE COLUMNS

In [27]:
df.groupBy("department","state")\
    .sum("salary","hike")\
    .show(truncate=False)

+----------+-----+-----------+---------+
|department|state|sum(salary)|sum(hike)|
+----------+-----+-----------+---------+
|Sales     |NY   |176000     |30000    |
|Sales     |CA   |81000      |23000    |
|Finance   |CA   |189000     |47000    |
|Finance   |NY   |162000     |34000    |
|Marketing |NY   |91000      |21000    |
|Marketing |CA   |80000      |18000    |
+----------+-----+-----------+---------+



RUNNING MORE AGGRGATE IN ONE TIME 

In [51]:
df.groupBy("department") \
.agg(
    sum("salary").alias("total_salary"),
    max("salary").alias("max_salary"),
    sum("hike").alias("total_hike"),
    max("hike").alias("max_hike")
    ) \
        .show()

+----------+------------+----------+----------+--------+
|department|total_salary|max_salary|total_hike|max_hike|
+----------+------------+----------+----------+--------+
|     Sales|      257000|     90000|     53000|   23000|
|   Finance|      351000|     99000|     81000|   24000|
| Marketing|      171000|     91000|     39000|   21000|
+----------+------------+----------+----------+--------+



In [41]:
from pyspark.sql.functions import sum, avg, max

df.groupBy("department") \
  .agg(
      sum("salary").alias("sum_salary"),
      avg("salary").alias("avg_salary"),
      sum("hike").alias("sum_bonus"),
      max("hike").alias("max_bonus")
  ) \
  .show(truncate=False)


+----------+----------+-----------------+---------+---------+
|department|sum_salary|avg_salary       |sum_bonus|max_bonus|
+----------+----------+-----------------+---------+---------+
|Sales     |257000    |85666.66666666667|53000    |23000    |
|Finance   |351000    |87750.0          |81000    |24000    |
|Marketing |171000    |85500.0          |39000    |21000    |
+----------+----------+-----------------+---------+---------+



USING WHERE CLAUSE ON AGGREGATE DATA

In [52]:
df.groupBy("department") \
.agg(
    sum("salary").alias("total_salary"),
    max("salary").alias("max_salary"),
    sum("hike").alias("total_hike"),
    max("hike").alias("max_hike")
    ) \
        .where(col("total_hike")>=9000)\
        .show()

+----------+------------+----------+----------+--------+
|department|total_salary|max_salary|total_hike|max_hike|
+----------+------------+----------+----------+--------+
|     Sales|      257000|     90000|     53000|   23000|
|   Finance|      351000|     99000|     81000|   24000|
| Marketing|      171000|     91000|     39000|   21000|
+----------+------------+----------+----------+--------+



USING SQL GROUP BY QUERY

In [57]:
df.createOrReplaceTempView("employees")

In [59]:
#  Using SQL Query
sql_string = """SELECT department,
       SUM(salary) AS sum_salary,
       AVG(salary) AS avg_salary,
       SUM(hike) AS sum_bonus,
       MAX(hike) AS max_bonus
FROM employees
GROUP BY department
HAVING SUM(hike) >= 50000"""

# Execute SQL query against the temporary view
df2 = spark.sql(sql_string)
df2.show()

+----------+----------+-----------------+---------+---------+
|department|sum_salary|       avg_salary|sum_bonus|max_bonus|
+----------+----------+-----------------+---------+---------+
|     Sales|    257000|85666.66666666667|    53000|    23000|
|   Finance|    351000|          87750.0|    81000|    24000|
+----------+----------+-----------------+---------+---------+

